# HerWay: Safest Route Finder using A* Algorithm

This notebook implements the A* search algorithm to find the **safest possible route** between two locations. The path is determined by finding the sequence of areas that results in the lowest cumulative 'danger score'.

**A* Implementation (`f(n) = g(n) + h(n)`):**
- **`g(n)` (Cost to Reach):** The cumulative **danger score** from the start to the current node `n`. The algorithm's primary goal is to minimize this value.
- **`h(n)` (Heuristic):** The straight-line geographical distance. This acts as a guide to ensure the algorithm finds an efficient path among equally safe options.

**Key Features:**
1.  **Safety-First Pathfinding:** The algorithm is hardcoded to always prioritize the path with the minimum total danger score.
2.  **Data-Driven Danger Score:** A `danger_score` is calculated for each area based on crime counts and streetlight quality.
3.  **Hop-by-Hop Visualization:** The final route is plotted on a `folium` map, showing all the intermediate areas (hops) from the start to the destination.

## 1. Setup and Library Installation

In [8]:
!pip install pandas geopy folium networkx ipywidgets openpyxl

## 2. Importing Libraries

In [9]:
import pandas as pd
import numpy as np
import folium
import networkx as nx
import heapq
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
from math import radians, sin, cos, sqrt, atan2
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings

warnings.filterwarnings('ignore')

## 3. Data Loading and Preprocessing

**Instructions:**
1.  Click on the folder icon in the left sidebar.
2.  Click on the 'Upload to session storage' button.
3.  Select the `HerWay_final_dataset.xlsx` file.

In [10]:
excel_file_path = 'HerWay_final_dataset.xlsx'

sheet_names_map = {
    'delhi_combined_data': 'Delhi',
    'mumbai_combined_data': 'Mumbai',
    'chennai_combined_data': 'Chennai',
    'bengaluru_combined_data': 'Bengaluru',
    'kochi_combined_data': 'Kochi'
}

all_dfs = []

try:
    xls = pd.read_excel(excel_file_path, sheet_name=list(sheet_names_map.keys()))
    for sheet_name, city_name in sheet_names_map.items():
        if sheet_name in xls:
            df = xls[sheet_name]
            df['city'] = city_name
            all_dfs.append(df)
            print(f"Successfully loaded {city_name} data from sheet '{sheet_name}'.")

    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        print("\nAll datasets combined successfully!")
    else:
        print("\nNo data was loaded. Please check the Excel file and sheet names.")

except FileNotFoundError:
    print(f"Error: '{excel_file_path}' not found. Please make sure you have uploaded the file.")
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")

Successfully loaded Delhi data from sheet 'delhi_combined_data'.
Successfully loaded Mumbai data from sheet 'mumbai_combined_data'.
Successfully loaded Chennai data from sheet 'chennai_combined_data'.
Successfully loaded Bengaluru data from sheet 'bengaluru_combined_data'.
Successfully loaded Kochi data from sheet 'kochi_combined_data'.

All datasets combined successfully!


### 3.1. Calculating the Danger Score

We will now calculate a `danger_score` for each area. A **higher score indicates more danger**. This score will be the primary cost for our A* algorithm.

In [11]:
crime_counts = combined_df.groupby(['city', 'area_name']).size().reset_index(name='crime_count')
area_features = combined_df.groupby(['city', 'area_name']).agg({
    'lighting_quality_score': 'mean',
    'uptime_ratio': 'mean'
}).reset_index()

processed_data = pd.merge(crime_counts, area_features, on=['city', 'area_name'])

def normalize(series):
    if series.max() == series.min():
        return pd.Series(0, index=series.index)
    return (series - series.min()) / (series.max() - series.min())

processed_data['crime_norm'] = processed_data.groupby('city')['crime_count'].transform(normalize)
processed_data['lighting_inverted_norm'] = 1 - processed_data.groupby('city')['lighting_quality_score'].transform(normalize)

crime_weight = 0.7
lighting_weight = 0.3

processed_data['danger_score'] = (crime_weight * processed_data['crime_norm']) + (lighting_weight * processed_data['lighting_inverted_norm'])

processed_data['danger_score'] = 1 + 9 * normalize(processed_data['danger_score'])

print("Danger scores calculated:")
display(processed_data.head())

Danger scores calculated:


,city,area_name,crime_count,lighting_quality_score,uptime_ratio,crime_norm,lighting_inverted_norm,danger_score
0,Bengaluru,BTM Layout,6,3.166667,0.965126,0.272727,0.580,4.592074
1,Bengaluru,Banashankari,6,3.166667,0.971496,0.272727,0.580,4.592074
2,Bengaluru,Bannerghatta,8,3.625000,0.980795,0.454545,0.195,4.707962
3,Bengaluru,Basavanagudi,7,3.142857,0.972632,0.363636,0.600,5.277557
4,Bengaluru,Bellandur,12,3.083333,0.969095,0.818182,0.650,8.557315


## 4. Geocoding Area Names

In [12]:
geolocator = Nominatim(user_agent="safest_route_finder_v6")
coordinates = {}

unique_areas = processed_data[['city', 'area_name']].drop_duplicates()

print("Starting geocoding process... This may take a while.")
for index, row in unique_areas.iterrows():
    city, area = row['city'], row['area_name']
    query = f"{area}, {city}, India"
    try:
        location = geolocator.geocode(query, timeout=10)
        coordinates[(city, area)] = (location.latitude, location.longitude) if location else None
        print(f"Geocoded: {query} -> {coordinates.get((city, area))}")
        time.sleep(1)
    except (GeocoderTimedOut, GeocoderUnavailable):
        print(f"Service timed out for {query}. Retrying...")
        time.sleep(5)
        try:
            location = geolocator.geocode(query, timeout=15)
            coordinates[(city, area)] = (location.latitude, location.longitude) if location else None
        except Exception as e:
            print(f"Could not geocode {query} after retry: {e}")
            coordinates[(city, area)] = None
    except Exception as e:
        print(f"An error occurred for {query}: {e}")
        coordinates[(city, area)] = None

processed_data['coords'] = processed_data.apply(lambda r: coordinates.get((r['city'], r['area_name'])), axis=1)
processed_data.dropna(subset=['coords'], inplace=True)
print(f"\nGeocoding complete.")

Starting geocoding process... This may take a while.
Geocoded: BTM Layout, Bengaluru, India -> (12.9140008, 77.6102821)
Geocoded: Banashankari, Bengaluru, India -> (12.9278196, 77.556621)
Geocoded: Bannerghatta, Bengaluru, India -> (12.9297508, 77.6005147)
Geocoded: Basavanagudi, Bengaluru, India -> (12.9417261, 77.5755021)
Geocoded: Bellandur, Bengaluru, India -> (12.9361208, 77.6671843)
Geocoded: CV Raman Nagar, Bengaluru, India -> (12.9850987, 77.6631173)
Geocoded: Cooke Town, Bengaluru, India -> (13.0021998, 77.6251814)
Geocoded: Domlur, Bengaluru, India -> (12.9624669, 77.6381958)
Geocoded: Electronic City, Bengaluru, India -> (12.8487599, 77.648253)
Geocoded: Frazer Town, Bengaluru, India -> (12.9975851, 77.6136744)
Geocoded: Girinagar, Bengaluru, India -> (12.9401439, 77.5444995)
Geocoded: HSR Layout, Bengaluru, India -> (12.9116225, 77.6388622)
Geocoded: Hebbal, Bengaluru, India -> (13.0382184, 77.5919)
Geocoded: Indiranagar, Bengaluru, India -> (12.9732913, 77.6404672)
Geocode

## 5. Graph Creation and A* Algorithm

### 5.1. Haversine Distance Function

In [13]:
def haversine_distance(coord1, coord2):
    """Calculate the distance between two lat/lon points in kilometers."""
    R = 6371.0
    lat1, lon1 = map(radians, coord1)
    lat2, lon2 = map(radians, coord2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

### 5.2. A* Algorithm (Optimized for Safety)

In [14]:
def a_star_search(graph, start_node, end_node, coords_map, danger_scores):
    open_set = []
    # Heap stores (f_score, g_score, node)
    # g_score is the cumulative danger score
    heapq.heappush(open_set, (0, 0, start_node))

    came_from = {}
    g_score = {node: float('inf') for node in graph.nodes}
    g_score[start_node] = 0

    f_score = {node: float('inf') for node in graph.nodes}
    # Initial f_score is just the heuristic from the start
    f_score[start_node] = haversine_distance(coords_map[start_node], coords_map[end_node])

    while open_set:
        _, current_g, current_node = heapq.heappop(open_set)

        if current_node == end_node:
            path = []
            while current_node in came_from:
                path.append(current_node)
                current_node = came_from[current_node]
            path.append(start_node)
            return path[::-1]

        for neighbor in graph.neighbors(current_node):
            # g(n) is the cumulative danger score
            tentative_g_score = current_g + danger_scores[neighbor]

            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current_node
                g_score[neighbor] = tentative_g_score

                # h(n) is the distance heuristic
                heuristic = haversine_distance(coords_map[neighbor], coords_map[end_node])

                f_score[neighbor] = tentative_g_score + heuristic
                heapq.heappush(open_set, (f_score[neighbor], tentative_g_score, neighbor))

    return None

### 5.3. Building the City Graphs

In [15]:
city_graphs = {}
city_coords = {}
city_danger_scores = {}

for city in processed_data['city'].unique():
    G = nx.Graph()
    city_df = processed_data[processed_data['city'] == city]
    nodes = city_df['area_name'].tolist()
    G.add_nodes_from(nodes)

    # Create a fully connected graph for each city
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            G.add_edge(nodes[i], nodes[j])

    city_graphs[city] = G
    city_coords[city] = pd.Series(city_df.coords.values, index=city_df.area_name).to_dict()
    city_danger_scores[city] = pd.Series(city_df.danger_score.values, index=city_df.area_name).to_dict()

print("Graphs for all cities have been created.")

Graphs for all cities have been created.


## 6. Visualization with Folium

In [16]:
def plot_safest_route(city, path, coords_map, danger_scores):
    if not path:
        print("No path found to display.")
        return

    avg_lat = np.mean([coords_map[loc][0] for loc in path])
    avg_lon = np.mean([coords_map[loc][1] for loc in path])

    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=12)

    # Add markers for all locations in the city
    for area, coord in coords_map.items():
        score = danger_scores.get(area, 'N/A')
        folium.Marker(
            location=coord,
            popup=f"{area}<br>Danger Score: {score:.2f}",
            icon=folium.Icon(color='gray', icon='info-sign')
        ).add_to(m)

    path_coords = [coords_map[loc] for loc in path]

    # Highlight path nodes and draw the route
    folium.Marker(location=path_coords[0], popup=f"START: {path[0]}", icon=folium.Icon(color='green', icon='play')).add_to(m)
    folium.Marker(location=path_coords[-1], popup=f"END: {path[-1]}", icon=folium.Icon(color='red', icon='stop')).add_to(m)

    for i in range(1, len(path) - 1):
        folium.Marker(
            location=path_coords[i],
            popup=f"HOP: {path[i]}<br>Danger Score: {danger_scores.get(path[i]):.2f}",
            icon=folium.Icon(color='blue', icon='circle')
        ).add_to(m)

    # Draw the line connecting all the hops
    folium.PolyLine(path_coords, color='purple', weight=3, opacity=0.8).add_to(m)

    return m

## 7. Interactive Route Finder

In [17]:
# --- Widgets --- #
city_dropdown = widgets.Dropdown(options=sorted(processed_data['city'].unique()), description='City:')
start_dropdown = widgets.Dropdown(description='Start:')
end_dropdown = widgets.Dropdown(description='End:')
find_button = widgets.Button(description="Find Safest Route")
output_area = widgets.Output()

# --- Logic --- #
def update_locations(change):
    selected_city = change['new']
    locations = sorted(processed_data[processed_data['city'] == selected_city]['area_name'].unique())
    start_dropdown.options = locations
    end_dropdown.options = locations

def on_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        city, start, end = city_dropdown.value, start_dropdown.value, end_dropdown.value

        if not all([city, start, end]):
            print("Please select a city, start, and end location.")
            return
        if start == end:
            print("Start and end locations cannot be the same.")
            return

        print(f"Finding the safest route from {start} to {end} in {city}...")

        graph, coords, scores = city_graphs[city], city_coords[city], city_danger_scores[city]

        # The danger weight slider is removed, so the function is called without it.
        path = a_star_search(graph, start, end, coords, scores)

        if path:
            total_dist = 0
            for i in range(len(path) - 1):
                total_dist += haversine_distance(coords[path[i]], coords[path[i+1]])

            total_danger = sum([scores[node] for node in path])

            print(f"\nSafest path found with intermediate hops:")
            print(f"{' -> '.join(path)}")
            print(f"\nTotal Route Distance: {total_dist:.2f} km")
            print(f"Total Danger Score: {total_danger:.2f}")

            route_map = plot_safest_route(city, path, coords, scores)
            display(route_map)
        else:
            print("Could not find a path.")

# --- Observers --- #
city_dropdown.observe(update_locations, names='value')
find_button.on_click(on_button_clicked)

update_locations({'new': city_dropdown.value})

# --- Display --- #
display(widgets.VBox([
    city_dropdown,
    start_dropdown,
    end_dropdown,
    find_button,
    output_area
]))

In [18]:
!pip install polyline


import requests
import polyline
import folium
import numpy as np

def get_osrm_route(coords_sequence):
    waypoints = [f"{lon},{lat}" for lat, lon in coords_sequence]
    coords_str = ";".join(waypoints)
    url = f"http://router.project-osrm.org/route/v1/driving/{coords_str}"
    params = {
        "overview": "full",
        "geometries": "polyline"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        route_polyline = data['routes'][0]['geometry']
        return polyline.decode(route_polyline)
    else:
        print(f"OSRM error: {response.status_code}")
        return None

def plot_osrm_route(city, path, coords_map, danger_scores):
    if not path:
        print("No path found to display.")
        return

    avg_lat = np.mean([coords_map[loc][0] for loc in path])
    avg_lon = np.mean([coords_map[loc][1] for loc in path])

    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=12)

    # All nodes in city
    for area, coord in coords_map.items():
        score = danger_scores.get(area, 'N/A')
        if score != 'N/A':
            if score < 3.5:
                color = "green"
            elif score < 6:
                color = "orange"
            else:
                color = "red"
        else:
            color = "gray"
        folium.CircleMarker(
            location=coord,
            radius=5,
            popup=f"{area}<br>Danger Score: {score:.2f}",
            color=color,
            fill=True,
            fill_opacity=0.7
        ).add_to(m)

    # Mark start and end
    folium.Marker(location=coords_map[path[0]], popup=f"START: {path[0]}", icon=folium.Icon(color='green', icon='play')).add_to(m)
    folium.Marker(location=coords_map[path[-1]], popup=f"END: {path[-1]}", icon=folium.Icon(color='red', icon='stop')).add_to(m)

    # OSRM route based on path coords
    path_coords = [coords_map[loc] for loc in path]
    osrm_route = get_osrm_route(path_coords)
    if osrm_route:
        folium.PolyLine(osrm_route, color='green', weight=7, opacity=0.9, dash_array="10,8").add_to(m)
    else:
        folium.PolyLine(path_coords, color='purple', weight=3, opacity=0.8).add_to(m)

    return m

# Modify your existing button callback to use this plotting

def on_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        city, start, end = city_dropdown.value, start_dropdown.value, end_dropdown.value

        if not all([city, start, end]):
            print("Please select a city, start, and end location.")
            return
        if start == end:
            print("Start and end locations cannot be the same.")
            return

        print(f"Finding the safest route from {start} to {end} in {city}...")

        graph, coords, scores = city_graphs[city], city_coords[city], city_danger_scores[city]

        path = a_star_search(graph, start, end, coords, scores)

        if path:
            total_dist = 0
            for i in range(len(path) - 1):
                from_coord, to_coord = coords[path[i]], coords[path[i + 1]]
                # sum distances along the path (optional)
                # you can also compute total danger or other metrics here

            print(f"\nSafest path found with intermediate hops:")
            print(" -> ".join(path))

            # Plot with OSRM route on folium
            route_map = plot_osrm_route(city, path, coords, scores)

            map_filename = 'safest_route.html'
            route_map.save(map_filename)
            print(f"\nMap has been saved as '{map_filename}'. Please open it to view the route.")

            display(route_map)
        else:
            print("Could not find a path. Try different locations.")

# Attach modified callback
find_button.on_click(on_button_clicked)
